In [ ]:
import torch
import torch.nn as nn

# Example of a convolutional layer
conv_layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)

UP SAMPLING

In [ ]:
import torch.nn.functional as F

# Example of up sampling
x = torch.randn(1, 16, 32, 32)  # Example input tensor
upsampled_x = F.interpolate(x, scale_factor=2, mode='nearest')  # Upsample by a factor of 2 (nearest: duplicating the existing pixels or samples to create new ones)


In [ ]:
import numpy as np
array = np.array([[[1,2],[2,3]],[[4,5],[6,7]], [[8,9],[10,11]]], dtype=np.float64)
print(array, array.shape)
interpolated = F.interpolate(torch.from_numpy(array), scale_factor=2, mode='nearest')
print(interpolated, interpolated.shape)

[[[ 1.  2.]
  [ 2.  3.]]

 [[ 4.  5.]
  [ 6.  7.]]

 [[ 8.  9.]
  [10. 11.]]] (3, 2, 2)
tensor([[[ 1.,  1.,  2.,  2.],
         [ 2.,  2.,  3.,  3.]],

        [[ 4.,  4.,  5.,  5.],
         [ 6.,  6.,  7.,  7.]],

        [[ 8.,  8.,  9.,  9.],
         [10., 10., 11., 11.]]], dtype=torch.float64) torch.Size([3, 2, 4])


DOWNSAMPLING

In [ ]:
# Example of downsampling using max pooling
max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
downsampled_x = max_pool(x)  # Down sample by a factor of 2
print(downsampled_x.shape)

torch.Size([1, 16, 16, 16])


BATCH NORMALIZATION

In [ ]:
# Example of batch normalization
batch_norm = nn.BatchNorm2d(num_features=16)
normalized_x = batch_norm(upsampled_x)


ACTIVATION FUNCTIONS

In [ ]:
# Example of ReLU activation function
relu = nn.ReLU()
activated_x = relu(normalized_x)


In [ ]:
activated_x.shape

torch.Size([1, 16, 64, 64])

LINEAR LAYERS

In [ ]:
# Example of a linear layer
linear_layer = nn.Linear(in_features=65536, out_features=10)
output = linear_layer(activated_x.view(activated_x.size(0), -1))  # Flatten the input before feeding into the linear layer

CREATING THE NEURAL NETWORK

1. DEFINE NETWORK

In [2]:
import torch.nn as nn
import torch.nn.functional as F
from google.colab import drive
import numpy as np
import pandas as pd
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm import tqdm  # For progress bar
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch.optim as optim

drive.mount('/content/drive')

Mounted at /content/drive


2. DEFINE LOSS FUNCTION AND OPTIMIZER

3. TRAIN THE NETWORK

4. TEST THE NETWORK

TRAINING WITH OUR DATA

In [ ]:
# Use ResNet50
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)  # Adjust the final layer for 5 classes

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
class APTOSDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform):
        self.data_frame = csv_file
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, 'train_images', self.data_frame.iloc[idx, 0] + '.png')
        image = Image.open(img_name).convert('RGB')
        label = self.data_frame.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
        return image, label





In [ ]:
data_path = '/content/drive/My Drive/aptos2019-blindness-detection'
csv_file = os.path.join(data_path, 'train.csv')

In [ ]:
df = pd.read_csv(csv_file)

In [ ]:
from sklearn.model_selection import train_test_split
X = df.iloc[:, :]
y = df.iloc[:, :]

train_set, test_set, ignore1, ignore2 = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(90),
    transforms.RandomApply([
        transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1))
    ], p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    #transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
trainset = APTOSDataset(csv_file = train_set, root_dir = data_path, transform=transform)
trainloader = DataLoader(trainset, batch_size=16, shuffle=True, num_workers = 2)

validset = APTOSDataset(csv_file = test_set, root_dir = data_path, transform=transform)
validloader = DataLoader(validset, batch_size=16, shuffle=False, num_workers=2)

In [ ]:
'''full_dataset = APTOSDataset(csv_file=csv_file, root_dir=data_path, transform=transform)
train_size = int(0.8 * len(full_dataset))  # 80% Train
val_size = int(0.1 * len(full_dataset))  # 10% Val
test_size = len(full_dataset) - train_size - val_size  # 10% Test'''

'full_dataset = APTOSDataset(csv_file=csv_file, root_dir=data_path, transform=transform)\ntrain_size = int(0.8 * len(full_dataset))  # 80% Train\nval_size = int(0.1 * len(full_dataset))  # 10% Val\ntest_size = len(full_dataset) - train_size - val_size  # 10% Test'

In [ ]:
def train(model, train_loader, valid_loader, criterion, optimizer, epochs=10):
    train_losses, valid_losses = [], []
    train_accuracies, valid_accuracies = [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        valid_loss, valid_accuracy = validate(model, valid_loader, criterion)
        valid_losses.append(valid_loss)
        valid_accuracies.append(valid_accuracy)

        print(f"Epoch {epoch+1}, Train Loss: {train_loss}, Train Accuracy: {train_accuracy}%, Valid Loss: {valid_loss}, Valid Accuracy: {valid_accuracy}%")

    return train_losses, valid_losses, train_accuracies, valid_accuracies

def validate(model, valid_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct, total = 0, 0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    valid_loss = running_loss / len(valid_loader)
    valid_accuracy = 100 * correct / total
    return valid_loss, valid_accuracy

print(trainloader)
# Train the model
train_losses, valid_losses, train_accuracies, valid_accuracies = train(model, trainloader, validloader, criterion, optimizer, epochs=10)

Epoch 1, Train Loss: 0.8137676811412625, Train Accuracy: 71.1847046773643%, Valid Loss: 0.7569992172977199, Valid Accuracy: 70.80491132332878%
Epoch 2, Train Loss: 0.699427691652723, Train Accuracy: 73.74530556503926%, Valid Loss: 0.6204840651024943, Valid Accuracy: 77.62619372442019%
Epoch 3, Train Loss: 0.6496905889362097, Train Accuracy: 76.51075452372824%, Valid Loss: 0.6742133446361708, Valid Accuracy: 73.66984993178717%
Epoch 4, Train Loss: 0.6148785328411538, Train Accuracy: 78.01297371116422%, Valid Loss: 0.6920323235833127, Valid Accuracy: 76.26193724420192%
Epoch 5, Train Loss: 0.6087981827719056, Train Accuracy: 77.46671218846022%, Valid Loss: 0.5768162517443948, Valid Accuracy: 78.85402455661665%
Epoch 6, Train Loss: 0.5917606612100549, Train Accuracy: 77.60327756913622%, Valid Loss: 0.5893783430042474, Valid Accuracy: 80.76398362892223%
Epoch 7, Train Loss: 0.5535876311199821, Train Accuracy: 78.93479003072721%, Valid Loss: 0.5619445682867713, Valid Accuracy: 81.4461118690

In [ ]:
with tqdm(total=len(train_loader), desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch") as pbar:

In [5]:
# Plotting training and validation loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(valid_losses, label='Valid Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plotting training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(valid_accuracies, label='Valid Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Confusion matrix
conf_matrix = confusion_matrix(test_labels.cpu(), torch.argmax(test_outputs, 1).cpu())
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=range(5), yticklabels=range(5))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
def make_confusion_matrix(train_tp, train_tn, train_fp, train_fn, labels=['No', 'Yes']):
    # Calculate metrics from TP, TN, FP, FN
    cm = np.array([[np.sum(train_tn), np.sum(train_fp)],
                   [np.sum(train_fn), np.sum(train_tp)]])
    # Create DataFrame for confusion matrix
    df_cm = pd.DataFrame(cm, index=[f'Actual - {labels[0]}', f'Actual - {labels[1]}'],
                         columns=[f'Predicted - {labels[0]}', f'Predicted - {labels[1]}'])
    # Create annotations for each cell
    group_counts = ["{0:0.0f}".format(value) for value in cm.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in cm.flatten() / np.sum(cm)]
    labels = [f"{v1}\n{v2}" for v1, v2 in zip(group_counts, group_percentages)]
    labels = np.asarray(labels).reshape(2, 2)
    # Plotting the heatmap with adjusted color scheme
    plt.figure(figsize=(10, 7))
    sns.heatmap(df_cm, annot=labels, fmt='', cmap='YlGnBu', cbar=False)  # Change 'cmap' to adjust the color scheme
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Train Confusion Matrix Across All Epochs')
    plt.show()
# Example usage:
make_confusion_matrix(train_tp, train_tn, train_fp, train_fn)

In [ ]:
dummy_input = torch.randn(1, 3, 224, 224)  # Replace 224 with your input size
dummy_output = model.conv2(model.pool(F.relu(model.batch_norm1(model.conv1(dummy_input)))))
dummy_output = model.pool(F.relu(model.batch_norm2(dummy_output)))
print(dummy_output.shape)  # This will help you determine the correct input size for fc1